In [15]:
import requests
import pandas as pd
from API_Key import api_key

pages = list(range(1,11))

popular = pd.DataFrame()

for page in pages:
    movieData = pd.DataFrame(requests.get(f'https://api.themoviedb.org/3/movie/popular?api_key={api_key}&language=en-US&page={page}').json()['results'])
    popular = popular.append(movieData)

popular.head()


popular = popular[popular['adult'] == False]

titleNid = popular[['id','title']]

titleNid = titleNid.reset_index(drop = True)

In [16]:
popular_features = popular[['id','title','release_date','popularity','vote_average','vote_count']]
popular_features = popular_features.rename(columns={
                                        "id":"movie_id",
                                        "popularity":"movie_popularity"
                                        })

In [17]:
titleNid = titleNid.rename(columns={"id" : 'movie_id'})

In [18]:
titleNid.to_csv('Tables/titleNid.csv',index = False)

In [19]:
# Loop through ids for revnue values
id_list = titleNid['movie_id'].tolist()

revenue_list = []
budget_list = []

for movie in id_list:
    x = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['revenue']
    y = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['budget']
    revenue_list.append(x)
    budget_list.append(y)

revenue_df = pd.DataFrame(
    {'movie_id':id_list,
    'budget': budget_list,
    'revenue':revenue_list
    })

revenue_df.reset_index(drop = True)

,movie_id,budget,revenue
0,566525,150000000,430238384
1,370172,242000000,708000000
2,580489,110000000,442000000
3,522402,0,0
4,512195,200000000,0
...,...,...,...
195,324786,40000000,175302354
196,410113,0,0
197,623135,0,0
198,874209,0,0


In [8]:
popular_features

,movie_id,title,release_date,movie_popularity,vote_average,vote_count
0,566525,Shang-Chi and the Legend of the Ten Rings,2021-09-01,8304.971,7.9,3094
1,370172,No Time to Die,2021-09-29,4398.328,7.6,1870
2,580489,Venom: Let There Be Carnage,2021-09-30,4128.057,6.8,1987
3,522402,Finch,2021-11-04,3098.869,8.2,986
4,512195,Red Notice,2021-11-04,3901.386,6.9,960
...,...,...,...,...,...,...
15,324786,Hacksaw Ridge,2016-10-07,197.352,8.2,10180
16,410113,The Loud House Movie,2021-08-20,173.402,8.4,86
17,623135,Sinkhole,2021-08-11,176.414,7.2,23
18,874209,Spider-Man: Responsibility,2016-09-23,190.281,4.9,4


In [9]:

Movie_Data = popular_features.merge(revenue_df, on='movie_id', how='inner')
Movie_Data.to_csv('Tables/Movie_Data.csv', index=False)